In [ ]:
# Phase 1 : Data Sanitation and Preprocessing
# =============================

import pandas as pd
import csv

# Load the dataset
df = pd.read_csv('online_retail.csv')

# Remove missing customers
df = df.dropna(subset=['Customer ID'])

# Convert 'Customer Id' to integer
df['Customer ID'] = df['Customer ID'].astype(int)

# Remove canceled orders
df = df[~df['Invoice'].str.startswith('C')]

# Remove rows where item price was '0'
df = df[df['Price'] > 0]

# Remove non-product stock codes
# List of non-product codes
non_product_codes = ['POST', 'DOT', 'ADJUST', 'C2', 'M', 'BANK CHARGES']

# Keep only rows where StockCode is NOT in the list
df = df[~df['StockCode'].isin(non_product_codes)]

# Create 'TotalPrice' column
df['TotalPrice'] = df['Quantity'] * df['Price']

# Convert 'InvoiceDate' to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Extract Year, Month, DayOfWeek, Hour:
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek
df['Hour'] = df['InvoiceDate'].dt.hour

# Save the cleaned dataset to a new CSV file
df.to_csv('online_retail_cleaned.csv', index=False)


In [ ]:
# Phase 2: Exploratory Data Analysis (EDA)
# =============================

import matplotlib.pyplot as plt
import seaborn as sns

# Visualization style
sns.set_theme(style="whitegrid", palette="muted")

# 1. Load Cleaned Dataset
df = pd.read_csv("online_retail_cleaned.csv")

# 2. Sales Performance & Seasonality

# Monthly revenue trends
monthly_sales = df.groupby(["Year","Month"])["TotalPrice"].sum().reset_index()
monthly_sales["YearMonth"] = monthly_sales["Year"].astype(str) + "-" + monthly_sales["Month"].astype(str)
plt.figure(figsize=(12,6))
sns.lineplot(
    data=monthly_sales,
    x="YearMonth",
    y="TotalPrice",
    marker="o",
    color="#2E86AB",
    linewidth=2.5
)
plt.xticks(rotation=45)
plt.title("Monthly Sales Revenue Over Time", fontsize=16, weight="bold")
plt.xlabel("Year-Month")
plt.ylabel("Revenue (£)")
plt.tight_layout()
plt.show()

# Day of week sales
plt.figure(figsize=(8,5))
sns.barplot(
    x="DayOfWeek",
    y="TotalPrice",
    data=df,
    estimator=sum
)
plt.title("Revenue by Day of the Week (0=Monday)", fontsize=16, weight="bold")
plt.xlabel("Day of Week")
plt.ylabel("Revenue (£)")
plt.tight_layout()
plt.show()

# Hourly sales
plt.figure(figsize=(8,5))
sns.barplot(
    x="Hour",
    y="TotalPrice",
    data=df,
    estimator=sum
)
plt.title("Revenue by Hour of Day", fontsize=16, weight="bold")
plt.xlabel("Hour")
plt.ylabel("Revenue (£)")
plt.tight_layout()
plt.show()


In [ ]:
# Geographic Footprint
# =============================

# Top 10 countries by revenue
country_sales = df.groupby("Country")["TotalPrice"].sum().nlargest(10)
plt.figure(figsize=(10,6))
sns.barplot(
    x=country_sales.index, 
    y=country_sales.values,
    hue=country_sales.index,
    palette="coolwarm"
)
plt.title("Top 10 Countries by Revenue", fontsize=16, weight="bold")
plt.xlabel("Country")
plt.ylabel("Revenue (£)")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

# UK vs International sales
uk_sales = df[df["Country"]=="United Kingdom"]["TotalPrice"].sum()
intl_sales = df[df["Country"]!="United Kingdom"]["TotalPrice"].sum()
plt.figure(figsize=(6,6))
colors = ["#1f77b4", "#ff7f0e"]
plt.pie(
    [uk_sales, intl_sales],
    labels=["UK", "International"],
    autopct="%.1f%%",
    startangle=90,
    colors=colors
)
plt.title("UK vs International Sales Share", fontsize=16, weight="bold")
plt.show()


In [ ]:
# Product Performance Analysis
# =============================

# Load dataset
df = pd.read_csv("online_retail_cleaned.csv")

# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Top 10 products by Quantity
top_products_qty = df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

print("=== Top 10 Products by Quantity ===")
print(top_products_qty)

plt.figure(figsize=(12,6))
sns.barplot(
    x=top_products_qty.index,
    y=top_products_qty.values,
    hue=top_products_qty.index,
    palette="Greens_d"
)
plt.title("Top 10 Products by Quantity", fontsize=16, weight="bold")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Quantity Sold")
plt.tight_layout()
plt.show()

# Top 10 products by Revenue
top_products_rev = df.groupby('Description')['TotalPrice'].sum().sort_values(ascending=False).head(10)

print("=== Top 10 Products by Revenue ===")
print(top_products_rev)

plt.figure(figsize=(12,6))
sns.barplot(
    x=top_products_rev.index,
    y=top_products_rev.values,
    hue=top_products_rev.index,
    palette="Reds_d"
)
plt.title("Top 10 Products by Revenue", fontsize=16, weight="bold")
plt.xticks(rotation=45, ha="right")
plt.ylabel("Revenue (£)")
plt.tight_layout()
plt.show()

# Merge for comparison
comparison = pd.DataFrame({
    "Quantity": top_products_qty,
    "Revenue": top_products_rev
}).fillna(0)

print("=== Comparison of Top Products by Quantity vs Revenue ===")
print(comparison)

# Plot side-by-side bars
comparison.plot(
    kind='bar',
    figsize=(14,6),
    color=["#2ecc71","#e74c3c"]
)
plt.title("Comparison: Top Products by Quantity vs Revenue", fontsize=16, weight="bold")
plt.ylabel("Values (Qty / Revenue)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Phase 3: Advanced Analytics - RFM Customer Segmentation
# ==============================

df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

snapshot_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)

rfm = df.groupby("Customer ID").agg({
     "InvoiceDate": lambda x: (snapshot_date - x.max()).days,
    "Invoice": "nunique",
    "TotalPrice": "sum"
}).rename(columns={"InvoiceDate":"Recency","Invoice":"Frequency","TotalPrice":"Monetary"})

fig, axes = plt.subplots(1,3, figsize=(18,5))
sns.histplot(rfm["Recency"], bins=30, ax=axes[0], color="#3498db")
axes[0].set_title("Recency Distribution")
sns.histplot(rfm["Frequency"], bins=30, ax=axes[1], color="#2ecc71")
axes[1].set_title("Frequency Distribution")
sns.histplot(rfm["Monetary"], bins=30, ax=axes[2], color="#e74c3c")
axes[2].set_title("Monetary Distribution")
plt.tight_layout()
plt.show()

In [ ]:
# Phase 4: Strategic Recommendations
# ==============================


#load the dataset
df = pd.read_csv('online_retail_cleaned.csv')

# Group data by customer and calculate total spending
customer_spending = df.groupby('Customer ID')['TotalPrice'].sum().reset_index()

# Rename column for clarity
customer_spending.rename(columns={'TotalPrice': 'TotalSpending'}, inplace=True)

# Quick check
customer_spending.head()

# Use a clean style
sns.set_style("whitegrid")

# Histogram
plt.figure(figsize=(10,6))
sns.histplot(
    customer_spending['TotalSpending'],
    bins=100,
    kde=True,
    color="#4C72B0"
)
plt.title("Customer Spending Distribution", fontsize=15, weight='bold')
plt.xlabel("Total Spending (£)")
plt.ylabel("Number of Customers")
plt.tight_layout()
plt.show()

# Boxplot
plt.figure(figsize=(10,3))
sns.boxplot(
    x=customer_spending['TotalSpending'],
    color="#DD8452")
plt.title("Customer Spending Boxplot", fontsize=15, weight='bold')
plt.xlabel("Total Spending (£)")
plt.tight_layout()
plt.show()

# Find threshold for top 10% spenders
threshold = customer_spending['TotalSpending'].quantile(0.90)

# Add a customer type column
customer_spending['CustomerType'] = customer_spending['TotalSpending'].apply(
    lambda x: 'Wholesaler' if x >= threshold else 'Retail'
)

# Count how many in each group
counts = customer_spending['CustomerType'].value_counts()
total_customers = len(customer_spending)
retail_count = counts.get('Retail', 0)
wholesale_count = counts.get('Wholesaler', 0)

# ----- Print Summary -----
print("Findings from Wholesaler Hypothesis\n")
print(f"Total Customers Analyzed: {total_customers}")
print(f"Retail Customers (≈90%): {retail_count}")
print(f"Wholesalers (≈10%): {wholesale_count}\n")

print("Observations")
print("- The spending distribution is highly right-skewed.")
print("- Majority of customers are low-spending Retail buyers.")
print("- A small but significant group of high-spending customers exists(Wholesalers).")
print("- Wholesalers, though fewer in number, contribute a disproportionate large share of total revenue.\n")

print("Business Strategy Recommendations")

print("1. Wholesalers (High-Value Segment)")
print(" - Provide bulk purchase discounts and tiered loyalty rewards.")
print(" - Offer priority services such as faster delivery and dedicated account managers.")
print(" - Focus on long-term contracts and exclusive product offerings to strengthen relationships.\n")

print("2. Retail Customers (Mass Market Segment)")
print(" - Implement seasonal promotions and time-limited discounts to encourage frequent purchases.")
print(" - Use personalized marketing campaigns (email/SMS) based on purchase history.")
print(" - Promote bundled deals (e.g., Buy More, Save More) to increase basket size.")


In [ ]:
# Phase 5: Data Enrichment via API Integration
# ==============================

import requests

# Load dataset
df = pd.read_csv("online_retail_cleaned.csv")

# API URL for GBP conversion
url = "https://open.er-api.com/v6/latest/GBP"

# Fetch exchange rates
response = requests.get(url)
data = response.json()
print(data)

# Extract rates
usd_rate = data['rates']['USD']
eur_rate = data['rates']['EUR']
print("1 GBP = ", usd_rate, "USD")
print("1 GBP = ", eur_rate, "EUR")

# Select top 100 transactions by TotalPrice
top100 = df.nlargest(100, 'TotalPrice').copy()
top100.head()

# Add converted columns
top100['TotalPrice_USD'] = top100['TotalPrice'] * usd_rate
top100['TotalPrice_EUR'] = top100['TotalPrice'] * eur_rate
top100[['Invoice', 'StockCode', 'Description', 'TotalPrice', 'TotalPrice_USD', 'TotalPrice_EUR']].head(10)